In [32]:
import pandas as pd

In [33]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructors = pd.read_csv('../data/constructors.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
drivers = pd.read_csv('../data/drivers.csv', parse_dates=[6])
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualifying = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv', parse_dates=[5])
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

In [34]:
result_with_dob = results.merge(drivers, left_on='driverId', right_on='driverId')
results_with_dob_date = result_with_dob.merge(races, on = 'raceId', how = 'left')

In [35]:
#results_with_dob_date['ageAtRace'] = pd.to_datetime(results_with_dob_date['date']) - pd.to_datetime(results_with_dob_date['dob'])
pd.to_datetime(results_with_dob_date['date'])
pd.to_datetime(results_with_dob_date['dob'])

0       1985-01-07
1       1985-01-07
2       1985-01-07
3       1985-01-07
4       1985-01-07
           ...    
24955   1995-06-29
24956   1995-06-29
24957   1995-09-23
24958   1996-06-25
24959   1996-06-25
Name: dob, Length: 24960, dtype: datetime64[ns]

In [36]:
#Calculate age at race in days (timedelta)
results_with_dob_date['ageAtRace'] = results_with_dob_date['date'] - results_with_dob_date['dob']

In [37]:
#Calculate how many days in f1 in days (timedelta)
date_of_debut = results_with_dob_date.groupby('driverId')['date'].min().to_frame()
date_of_debut = date_of_debut.merge(drivers, on = 'driverId', how = 'left').rename(columns={'date':'date_of_debut'})
date_of_debut = date_of_debut[['driverId','date_of_debut']]
date_of_debut['date_of_debut']
df = results_with_dob_date.merge(date_of_debut, on = 'driverId', how = 'left')
df['ageAtDebut'] = df['date_of_debut'] - df['dob']


In [44]:
#Drop columns
df = df.drop(columns=['number_y', 'number_x', 'code', 'forename', 'surname', 'dob', 'url_x', 'time_y', 'url_y'])

In [46]:
#Rename Columns
df = df.rename(columns={"grid": "gridStart", "name": "cirtcuitName"})

In [49]:
df.head()

,resultId,raceId,driverId,constructorId,gridStart,position,positionText,positionOrder,points,laps,...,driverRef,nationality,year,round,circuitId,cirtcuitName,date,ageAtRace,date_of_debut,ageAtDebut
0,1,18,1,1,1,1,1,1,10.0,58,...,hamilton,British,2008,1,1,Australian Grand Prix,2008-03-16,8469 days,2007-03-18,8105 days
1,27,19,1,1,9,5,5,5,4.0,56,...,hamilton,British,2008,2,2,Malaysian Grand Prix,2008-03-23,8476 days,2007-03-18,8105 days
2,57,20,1,1,3,13,13,13,0.0,56,...,hamilton,British,2008,3,3,Bahrain Grand Prix,2008-04-06,8490 days,2007-03-18,8105 days
3,69,21,1,1,5,3,3,3,6.0,66,...,hamilton,British,2008,4,4,Spanish Grand Prix,2008-04-27,8511 days,2007-03-18,8105 days
4,90,22,1,1,3,2,2,2,8.0,58,...,hamilton,British,2008,5,5,Turkish Grand Prix,2008-05-11,8525 days,2007-03-18,8105 days


In [50]:
#Add is racing at home feature
df['racingAtHome'] = df.apply(lambda x: x.nationality in x.circuitName, axis=1)

AttributeError: 'Series' object has no attribute 'circuitName'